In [1]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install scikit-learn
!pip install sentencepiece
!pip install sacremoses
!pip install evaluate

/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "


In [2]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [3]:
import pandas as pd

ds = pd.read_csv("CleanData/all_data_clean.csv")

texts = ds["Text"].tolist()
labels = ds["Label"].tolist()
languages = ds["Language"].tolist()


In [4]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)


In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('FineTuned_Model/xlm_15/', local_files_only=True)

In [7]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [8]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [10]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

import numpy as np

training_args = TrainingArguments(
    output_dir='./results/cleanData/xlm_15',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs/cleandata/',            # directory for storing logs
    logging_steps=10,
    # use_mps_device=True
)

model = AutoModelForSequenceClassification.from_pretrained("FineTuned_Model/xlm_15/")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    
)

predictions = trainer.predict(val_dataset)

preds = np.argmax(predictions.predictions, axis=-1)

metric.compute(predictions=preds, references=predictions.label_ids)




PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file FineTuned_Model/xlm_15/config.json
Model config XLMConfig {
  "_name_or_path": "FineTuned_Model/xlm_15/",
  "architectures": [
    "XLMForSequenceClassification"
  ],
  "asm": false,
  "attention_dropout": 0.1,
  "bos_index": 0,
  "bos_token_id": 0,
  "causal": false,
  "dropout": 0.1,
  "emb_dim": 1024,
  "embed_init_std": 0.02209708691207961,
  "end_n_top": 5,
  "eos_index": 1,
  "gelu_activation": true,
  "id2lang": {
    "0": "ar",
    "1": "bg",
    "10": "th",
    "11": "tr",
    "12": "ur",
    "13": "vi",
    "14": "zh",
    "2": "de",
    "3": "el",
    "4": "en",
    "5": "es",
    "6": "fr",
    "7": "hi",
    "8": "ru",
    "9": "sw"
  },
  "init_std": 0.02

{'accuracy': 0.4432399879554351}